In [1]:
import pandas as pd

videos_per_topic_docs_df = pd.read_csv("videos_per_topic_docs.csv")
video_id_to_doc_mapping_df = pd.read_csv("video_id_to_doc_mapping.csv")

video_id_to_doc_mapping_df


,Video Id,Video Title,Doc Name
0,--8n6A8Q6M0,$200 Luxury Beach Hotel in The Philippines 🇵🇭,doc0
1,-1B7cVoZr1c,Marine reacts to the Philippine Light Reaction...,doc1
2,-7vF5F-1btE,Ultimate Filipino Food Festival In The Netherl...,doc2
3,-9bfDHHneyU,SHOWING MY SISTER SB19 'GENTO' Music Video,doc3
4,-C5iB25BRsA,10 Reasons/Do not Retire TO the Philippines/Mo...,doc4
...,...,...,...
2699,zm_8N4vFnZw,PHILIPPINES PROVINCE LIFE IS SO DIFFERENT FROM...,doc2699
2700,zpT46etTP6E,"Starring real-life gay couple, latest Filipino...",doc2700
2701,zqvDHfgxWh8,🇰🇷Koreans React to P-pop Idol Group | Lovey Do...,doc2701
2702,zvTP6wl9sTU,FOREIGNER LIVING IN YHE PHILIPPINES WEEKLY FO...,doc2702


In [2]:
videos_per_topic_docs_df

,Topic,Custom Topic Label,Video_Ids
0,-1,Topic -1: Outliers,"['doc178', 'doc358', 'doc495', 'doc685', 'doc7..."
1,0,Topic 0: Delicious Dishes and Dining,"['doc0', 'doc2', 'doc3', 'doc4', 'doc5', 'doc7..."
2,1,Topic 1: Learning and Speaking Foreign Language,"['doc23', 'doc34', 'doc36', 'doc40', 'doc48', ..."
3,2,Topic 2: Christmas Season & Celebration,"['doc5', 'doc19', 'doc23', 'doc32', 'doc35', '..."
4,3,Topic 3: Experiences of Children in Community ...,"['doc9', 'doc22', 'doc24', 'doc70', 'doc127', ..."
...,...,...,...
117,116,Topic 116: Chicken & its Specific Related Dishes,"['doc210', 'doc263', 'doc277', 'doc323', 'doc3..."
118,117,Topic 117: Filipino Phrases,"['doc18', 'doc29', 'doc84', 'doc148', 'doc282'..."
119,118,Topic 118: Mommy and Baby's Morning Events,"['doc27', 'doc44', 'doc76', 'doc162', 'doc244'..."
120,119,Topic 119: Common Filipino Dishes,"['doc26', 'doc323', 'doc367', 'doc436', 'doc59..."
